# 4장. 마켓 타이밍 전략

In [ ]:
# coding: utf-8
from typing import Dict, Optional

import pandas as pd
import plotly.express as px

from data.data_loader import PykrxDataLoader
from simulation.account import Account
from simulation.broker import Broker
from simulation.utility import get_lookback_fromdate, rebalance
from simulation.metric import cagr, mdd, sharpe_ratio, sortino_ratio

from simulation.visualize import (plot_cumulative_return, plot_single_period_return,
                                  plot_relative_single_period_return,
                                  plot_cumulative_asset_profit, plot_asset_weight)


## 4.1. 이동 평균 전략 구현

### 4.1.1 이동 평균 최적화

In [ ]:
def calculate_moving_average(ohlcv_data: pd.DataFrame, period: int, ma_type: str) -> pd.DataFrame:
    # (1) 종가 가져오기
    close_data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)
    close_data = close_data['close']

    # (2) 이동 평균 값 계산하기
    if ma_type == 'sma':
        ma = close_data.rolling(window=period).mean()
    elif ma_type == 'ema':
        ma = close_data.ewm(span=period).mean()
    else:
        raise ValueError

    return ma

def get_moving_average_weights(ohlcv_data: pd.DataFrame, ma_data: pd.DataFrame) -> Optional[Dict]:
    # (1) 이동 평균 데이터 중 결측치가 있는지 확인함
    if ma_data.isnull().values.any():
        return None

    # (2) 매수할 주식과 매도할 주식을 선정함
    weights = {}
    stocks_to_buy = []
    for ticker in ohlcv_data['ticker']:
        # 종가 > 이동 평균
        if ohlcv_data.loc[ohlcv_data['ticker'] == ticker, 'close'].values > ma_data[ticker]:
            stocks_to_buy.append(ticker)
        # 종가 <= 이동 평균
        else:
            weights[ticker] = 0.0

    # 매수할 주식이 없는 경우 포트폴리오 반환
    if not stocks_to_buy:
        return weights

    # (3) 매수할 주식 비율을 할당함
    ratio = 1 / len(stocks_to_buy)
    for ticker in stocks_to_buy:
        weights[ticker] = ratio

    return weights

### 4.1.2 이동 평균 시뮬레이션

In [5]:
def simulate_moving_average(ohlcv_data: pd.DataFrame,
                            ma_type: str,
                            period: int) -> Account:
    # (1) 계좌 및 브로커 생성
    account = Account(initial_cash=100000000)
    broker = Broker()

    # (2) 이동 평균 값 계산
    ma = calculate_moving_average(ohlcv_data=ohlcv_data,period=period, ma_type=ma_type)
    print(ma)
    for date, ohlcv in ohlcv_data.groupby('date'):
        print(date.date())

        # (3) 주문 집행 및 계좌 갱신
        transactions = broker.process_order(dt=date, data=ohlcv, orders=account.orders)
        account.update_position(transactions=transactions)
        account.update_portfolio(dt=date, data=ohlcv)
        account.update_order()

        # (4) 이동 평균 전략을 이용해 포트폴리오 구성
        ma_slice = ma.loc[date]
        weights = get_moving_average_weights(ohlcv_data=ohlcv, ma_data=ma_slice)

        print(f'Portfolio: {weights}')
        if weights is None:
            continue

        # (5) 주문 생성
        rebalance(dt=date, data=ohlcv, account=account, weights=weights)

    return account


### 4.1.3. 시뮬레이션 실행

In [30]:
# 데이터 시작과 끝 날자 정의
fromdate = '2018-07-10'
todate = '2023-09-27'

# 투자할 종목 후보 정의
ticker_list = ['005930', '000660', '207940',
               '051910', '006400', '005380',
               '000270', '005490', '035420']

# 이동 평균 기간 정의
period = 3

# 기간을 고려한 데이터 시작 날짜 가져오기
adj_fromdate = get_lookback_fromdate(fromdate=fromdate, lookback=period, freq='m')

# 데이터 불러오기
data_loader = PykrxDataLoader(fromdate=adj_fromdate, todate=todate, market='KOSPI')
olcv_data = data_loader.load_stock_data(ticker_list=ticker_list, freq='m', delay=1)

# 데이터 확인하기
olcv_data.head()

/opt/anaconda3/envs/pyra312/lib/python3.12/site-packages/pykrx/stock/stock_api.py:50: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/Users/kms/dev/PyRA/data/data_loader.py:116: FutureWarning:

'm' is deprecated and will be removed in a future version, please use 'ME' instead.



,ticker,open,high,low,close,volume
date,,,,,,
2018-05-31,000270,31750,31900,31150,31150,1468344
2018-06-30,000270,31100,34000,30450,30850,18456227
2018-07-31,000270,30850,32850,29450,31700,17534882
2018-08-31,000270,31650,32600,30950,32050,13377137
2018-09-30,000270,31900,35600,31350,35100,19573161


In [6]:
# 이동 평균 종류 정의
ma_type = 'sma' # {ema, sma}

# 단순 이동 평균 전략 실행하기
sma_simulation_account = simulate_moving_average(ohlcv_data=olcv_data, ma_type=ma_type, period=period)

ticker            000270         000660         005380         005490  \
date                                                                    
2018-05-31           NaN            NaN            NaN            NaN   
2018-06-30           NaN            NaN            NaN            NaN   
2018-07-31  31233.333333   88466.666667  131333.333333  333000.000000   
2018-08-31  31533.333333   85000.000000  126666.666667  328666.666667   
2018-09-30  32950.000000   80800.000000  128000.000000  317166.666667   
...                  ...            ...            ...            ...   
2023-05-31  83800.000000   95566.666667  194000.000000  368333.333333   
2023-06-30  86300.000000  104433.333333  201333.333333  375000.000000   
2023-07-31  85700.000000  115733.333333  200833.333333  463333.333333   
2023-08-31  83800.000000  120133.333333  197200.000000  536333.333333   
2023-09-30  81433.333333  119966.666667  192066.666667  585333.333333   

ticker            005930         006400         03

In [22]:
# 이동 평균 종류 정의
ma_type = 'ema' # {ema, sma}

# 단순 이동 평균 전략 실행하기
ema_simulation_account = simulate_moving_average(ohlcv_data=olcv_data, ma_type=ma_type, period=period)

ticker            000270         000660         005380         005490  \
date                                                                    
2018-05-31  31150.000000   93400.000000  139000.000000  339500.000000   
2018-06-30  30950.000000   88266.666667  130000.000000  332500.000000   
2018-07-31  31378.571429   87142.857143  129714.285714  331357.142857   
2018-08-31  31736.666667   84933.333333  127200.000000  328766.666667   
2018-09-30  33472.580645   78825.806452  128387.096774  311080.645161   
...                  ...            ...            ...            ...   
2023-05-31  83013.299307   98614.775805  193718.846542  358054.388189   
2023-06-30  85756.649653  106907.387903  200109.423271  373027.194095   
2023-07-31  84228.324827  115153.693951  198054.711636  507513.597047   
2023-08-31  82214.162413  118476.846976  193577.355818  543256.798524   
2023-09-30  81807.081207  116588.423488  192338.677909  539128.399262   

ticker            005930         006400         03

## 2. 시뮬레이션 분석

### 2.1. 시뮬레이션 결과 전처리

#### 2.1.1. 히스토리 형식 변환

In [24]:
from simulation.utility import ticker_to_name, get_lookback_fromdate

sma_account = pd.DataFrame(sma_simulation_account.account_history).set_index('date')
sma_portfolio = pd.DataFrame(sma_simulation_account.portfolio_history).set_index('date')

ema_account = pd.DataFrame(ema_simulation_account.account_history).set_index('date')
ema_portfolio = pd.DataFrame(ema_simulation_account.portfolio_history).set_index('date')

#### 2.1.2. 룩백 기간 제거

In [23]:
analysis_fromdate = df_account.index[period]

#### 2.1.3. 단기 수익률 계산

In [25]:
sma_returns = sma_account['total_asset'].pct_change().loc[analysis_fromdate:]
sma_returns.name = 'sma_return'
sma_returns.head()

date
2018-08-31    0.002461
2018-09-30    0.039207
2018-10-31   -0.183878
2018-11-30    0.002911
2018-12-31    0.000000
Name: sma_return, dtype: float64

In [26]:
ema_returns = ema_account['total_asset'].pct_change().loc[analysis_fromdate:]
ema_returns.name = 'ema_return'
ema_returns.head()

date
2018-08-31    0.005688
2018-09-30    0.039048
2018-10-31   -0.155739
2018-11-30    0.003926
2018-12-31    0.000000
Name: ema_return, dtype: float64

#### 2.1.4. KOSPI 월간 수익률 계산

In [27]:
kospi = data_loader.load_index_data(ticker_list=['1001'], freq='m', delay=1)
kospi_returns = kospi['close'].pct_change().loc[analysis_fromdate:]
kospi_returns.iloc[0] = 0.0
kospi_returns.name = 'kospi_return'
kospi_returns.index.name = 'date'
kospi_returns.head()

/opt/anaconda3/envs/pyra312/lib/python3.12/site-packages/pykrx/stock/stock_api.py:50: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



date
2018-08-31    0.000000
2018-09-30    0.008692
2018-10-31   -0.133748
2018-11-30    0.033094
2018-12-31   -0.026621
Freq: ME, Name: kospi_return, dtype: float64

In [28]:
pd.concat([sma_returns, ema_returns, kospi_returns], axis=1)

,sma_return,ema_return,kospi_return
date,,,
2018-08-31,0.002461,0.005688,0.000000
2018-09-30,0.039207,0.039048,0.008692
2018-10-31,-0.183878,-0.155739,-0.133748
2018-11-30,0.002911,0.003926,0.033094
2018-12-31,0.000000,0.000000,-0.026621
...,...,...,...
2023-05-31,0.030800,0.035482,0.030218
2023-06-30,-0.001727,0.007031,-0.004982
2023-07-31,0.112955,0.114756,0.026635


### 2.2. 포트폴리오 성능 지표

#### 2.2.1. 연평균 복리 성장률(CAGR)

In [29]:
print(f'연평균 복리 성장률(단순이동평균): {cagr(returns=sma_returns, freq='m'): .3f}')
print(f'연평균 복리 성장률(지수이동평균): {cagr(returns=ema_returns, freq='m'): .3f}')
print(f'연평균 복리 성장률(KOSPI): {cagr(returns=kospi_returns, freq='m'): .3f}')

연평균 복리 성장률(단순이동평균):  0.062
연평균 복리 성장률(지수이동평균):  0.080
연평균 복리 성장률(KOSPI):  0.012


#### 2.2.2. 최대손실낙폭(MDD)

In [31]:
print(f'최대손실낙폭(단순이동평균): {mdd(returns=sma_returns): .3f}')
print(f'최대손실낙폭(지수이동평균): {mdd(returns=ema_returns): .3f}')
print(f'최대손실낙폭(KOSPI): {mdd(returns=kospi_returns): .3f}')

최대손실낙폭(단순이동평균): -0.462
최대손실낙폭(지수이동평균): -0.447
최대손실낙폭(KOSPI): -0.346


#### 2.2.3. 샤프 비율(Sharpe Ratio)

In [32]:
print(f'샤프 비율(단순이동평균): {sharpe_ratio(returns=sma_returns, freq='m'): .3f}')
print(f'샤프 비율(지수이동평균): {sharpe_ratio(returns=ema_returns, freq='m'): .3f}')
print(f'샤프 비율(KOSPI): {sharpe_ratio(returns=kospi_returns, freq='m'): .3f}')

샤프 비율(단순이동평균):  0.362
샤프 비율(지수이동평균):  0.431
샤프 비율(KOSPI):  0.158


#### 2.2.4. 소티노 비율(Sortino Ratio)

In [33]:
print(f'소티노 비율(단순이동평균): {sortino_ratio(returns=sma_returns, freq='m'): .3f}')
print(f'소티노 비율(지수이동평균): {sortino_ratio(returns=ema_returns, freq='m'): .3f}')
print(f'소티노 비율(KOSPI): {sortino_ratio(returns=kospi_returns, freq='m'): .3f}')

소티노 비율(단순이동평균):  0.356
소티노 비율(지수이동평균):  0.432
소티노 비율(KOSPI):  0.142


### 2.3. 시각화를 통한 시뮬레이션 분석

#### 2.3.1. 누적 수익률 곡선

In [35]:
# 누적 수익률 계산하기
benchmark_cum_returns = (kospi_returns + 1).cumprod() - 1
sma_cum_returns = (sma_returns + 1).cumprod() - 1
ema_cum_returns = (ema_returns + 1).cumprod() - 1

# 자산 정보 결합하기
cum_returns = pd.concat([benchmark_cum_returns, sma_cum_returns, ema_cum_returns], axis=1)
cum_returns.columns = ['코스피(KOSPI)', '단순이동평균(SMA)', '지수이동평균(EMA)']

# 자산 편화 시각화하기
fig = px.line(data_frame=cum_returns)

# x축, y축 레이블 설정
fig.update_xaxes(title_text='날짜')
fig.update_yaxes(title_text='누적 수익률')

# 범례 제목 설정
fig.update_layout(legend_title_text='포트폴리오')

fig.show()